I really liked Andrej Karpathy's article on Software 2.0 (https://medium.com/@karpathy/software-2-0-a64152b37c35) as it really gave a vision of what the future of software development could look like with tools like deep learning in the picture. Pete Warden had a follow-up to Andrej's post about it going into even more possibilities on the subject (https://petewarden.com/2017/11/13/deep-learning-is-eating-software/) and that got me thinking about how some of the simple things programmers do could be automated. 

The first item I thought of on that list was writing an email validator. Email validation turns out to be very tricky to get perfect but most programmers could come up with something that would be sufficient for the majority of cases.

Note: it looks like there's no longer a free download of the Alexa 1M top sites, so for domains, I'll pick them from the [Majestic Million](https://blog.majestic.com/development/majestic-million-csv-daily/).

In [1]:
import random
import string

import scipy.stats

def random_letters(length=10):
    """Generates random letters with a certain length"""
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(length))

def fake_email_generator(n=10):
    """Generates a fake 'valid-looking' email address"""
    
    domains = ['gmail.com', 'yahoo.com', 'aol.com', 'hotmail.com',
              'live.com', 'mail.ru']
    for _ in range(n):
        l = scipy.stats.poisson(9).rvs() + 1  # Strictly positive Poisson with mean 10
        domain = random.choice(domains)
        yield random_letters(l) + '@' + domain
        
def fake_non_email_generator(n=10):
    """Generates a fake string that isn't an email address"""
    for _ in range(n):
        l = scipy.stats.poisson(9).rvs() + 1
        yield random_letters(l)

In [2]:
print(list(fake_email_generator()))
print(list(fake_non_email_generator()))

['rpqlkssdapdaxwqmsc@aol.com', 'odwwohwtjc@yahoo.com', 'fociqvyzk@mail.ru', 'spaxeres@gmail.com', 'gxjbwt@hotmail.com', 'tzodige@gmail.com', 'vlighecxss@live.com', 'bgtyumzk@mail.ru', 'ykzcjesfiii@yahoo.com', 'omytwerxyfzbysm@yahoo.com']
['wummyupw', 'foqphyljepyofq', 'tgzfvidt', 'qsduzjzp', 'znyjexojvt', 'jwhmffyewu', 'njsfvscwidd', 'oupneoony', 'wfmrmr', 'cnrugyhvbjb']


In [3]:
import numpy as np

n_fake, n_real = 100, 50

raw_email_strings = list(fake_non_email_generator(n_fake)) + list(fake_email_generator(n_real))
y = np.concatenate([np.zeros((n_fake,)), np.ones((n_real,))])

In [4]:
# Perform some random shuffling of the data
# NB: shuffling needs to be the same between strings and y

shuffle_idx = np.arange(len(raw_email_strings))
np.random.shuffle(shuffle_idx)
shuffle_idx


raw_email_strings = [
    raw_email_strings[i] for i in shuffle_idx
]
y = y[shuffle_idx]

In [5]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tzr = Tokenizer(filters='', char_level=True)
tzr.fit_on_texts(raw_email_strings)
email_sequences = pad_sequences(tzr.texts_to_sequences(raw_email_strings))

Using TensorFlow backend.


In [6]:
num_characters = len(tzr.index_docs) + 1

In [7]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Embedding, LSTM

In [8]:
model = Sequential()
model.add(Embedding(num_characters, 10))
model.add(LSTM(200))
model.add(Dense(200, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [9]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', 
              metrics=['acc'])

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 10)          290       
_________________________________________________________________
lstm_1 (LSTM)                (None, 200)               168800    
_________________________________________________________________
dense_1 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 209,491
Trainable params: 209,491
Non-trainable params: 0
_________________________________________________________________


In [11]:
history = model.fit(email_sequences,
                    y,
                    batch_size=10,
                    epochs=10,
                    validation_split=0.2)

Train on 120 samples, validate on 30 samples
Epoch 1/10
120/120 [==============================] - 1s 6ms/step - loss: 0.9809 - acc: 0.6833 - val_loss: 0.5424 - val_acc: 0.9333
Epoch 2/10
120/120 [==============================] - 1s 4ms/step - loss: 0.4537 - acc: 0.8833 - val_loss: 1.1964 - val_acc: 0.3000
Epoch 3/10
120/120 [==============================] - 0s 4ms/step - loss: 0.5035 - acc: 0.8000 - val_loss: 0.8767 - val_acc: 0.3333
Epoch 4/10
120/120 [==============================] - 0s 4ms/step - loss: 0.5238 - acc: 0.7583 - val_loss: 0.5198 - val_acc: 0.8333
Epoch 5/10
120/120 [==============================] - 0s 3ms/step - loss: 0.3173 - acc: 0.9500 - val_loss: 0.1165 - val_acc: 0.9333
Epoch 6/10
120/120 [==============================] - 0s 3ms/step - loss: 0.2019 - acc: 0.9500 - val_loss: 0.0867 - val_acc: 0.9333
Epoch 7/10
120/120 [==============================] - 0s 3ms/step - loss: 0.1942 - acc: 0.9333 - val_loss: 0.2465 - val_acc: 0.9667
Epoch 8/10
120/120 [===========

In [12]:
test_email = ['test@gmail.com',
              'another.example@yahoo.com',
              'example@',
              '@example.com',
              'testtesttesttesttesttesttest',
              'gmail.com',
              'matt!@whatsup.net',
              'jason@mindspring.org',
              'blah@gatech.edu',
              '@yahoo.com']
test_seq = tzr.texts_to_sequences(test_email)

In [13]:
test_seq = pad_sequences(test_seq, maxlen=email_sequences.shape[-1])

In [14]:
np.set_printoptions(formatter={'float': '{:2f}'.format})
model.predict(test_seq)

array([[0.827229],
       [0.955771],
       [0.199287],
       [0.784823],
       [0.405493],
       [0.630595],
       [0.382036],
       [0.727012],
       [0.700789],
       [0.682027]], dtype=float32)